In [1]:
import os
from tensorflow.examples.tutorials.mnist import input_data
from google.colab import drive
drive.mount('data')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at data


In [0]:
import tensorflow as tf
import numpy as np

In [3]:
data_root = 'data/My Drive'
mnist_home = os.path.join(data_root, 'mnist')
mnist = input_data.read_data_sets(mnist_home, one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/My Drive/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/My Drive/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/My Drive/mnist/t10k-images-idx3-ubyte.gz
Extracting data/My Drive/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
X_train = mnist.train.images
X_test = mnist.test.images
Y_train = mnist.train.labels
Y_test = mnist.test.labels
num_outputs = 10 # 0-9 digits
num_inputs = 784 # total pixels

In [0]:
def mlp(x, num_inputs, num_outputs, num_layers, num_neurons):
  w = []
  b = []
  
  for i in range(num_layers):
    w.append(tf.Variable(tf.random_normal(
        [num_inputs if i==0 else num_neurons[i-1], num_neurons[i]]),
        name="w_{0:04d}".format(i)))
    b.append(tf.Variable(tf.zeros([num_neurons[i-1]]),
        name="b_{0:04d}".format(i)))
    
  w.append(tf.Variable(tf.random_normal(
    [num_neurons[num_layers - 1] if num_layers > 0 else num_inputs,
    num_outputs]), name="w_out"))
  b.append(tf.Variable(tf.random_normal([num_outputs]),
    name="b_out"))
      
  layer = x
  
  for i in range(num_layers):
    layer = tf.matmul(layer, w[i]) + b[i]
    layer = tf.nn.relu(layer)
    
  layer = tf.matmul(layer, w[num_layers]) + b[num_layers]
  
  return layer
    
  

In [0]:
def mnist_batch_func(batch_size=100):
  X_batch, Y_batch = mnist.train.next_batch(batch_size)
  return [X_batch, Y_batch]

In [0]:
with tf.Session() as tfs:
  tf.global_variables_initializer().run()

In [0]:
def tensorflow_classification(n_epochs, n_batches,
                              batch_size, batch_func,
                              model, optimizer, loss, accuracy_function,
                              X_test, Y_test):
  with tf.Session() as tfs:
    tfs.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
      epoch_loss = 0.0
      for batch in range(n_batches):
        X_batch, Y_batch = batch_func(batch_size)
        feed_dict = {x: X_batch, y: Y_batch}
        _, batch_loss = tfs.run([optimizer, loss], feed_dict)
        epoch_loss += batch_loss
      average_loss = epoch_loss / n_batches
      print("epoch: {0:04d} loss = {1:0.6f}".format(epoch, average_loss))
    feed_dict = {x: X_test, y: Y_test}
    accuracy_score = tfs.run(accuracy_function, feed_dict=feed_dict)
    print("accuracy={0:.8f}".format(accuracy_score))

In [0]:
# input images
x = tf.placeholder(dtype=tf.float32, name="x",
shape=[None, num_inputs])
# target output
y = tf.placeholder(dtype=tf.float32, name="y",
shape=[None, num_outputs])
num_layers = 2
num_neurons = [256,256]
learning_rate = 0.01
n_epochs = 50
batch_size = 100
n_batches = int(mnist.train.num_examples/batch_size)

In [0]:
model = mlp(x=x,num_inputs=num_inputs,num_outputs=num_outputs,
            num_layers=num_layers,num_neurons=num_neurons)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
predictions_check = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy_function = tf.reduce_mean(tf.cast(predictions_check, tf.float32))

In [11]:
tensorflow_classification(n_epochs=n_epochs,
  n_batches=n_batches,
  batch_size=batch_size,
  batch_func=mnist_batch_func,
  model = model,
  optimizer = optimizer,
  loss = loss,
  accuracy_function = accuracy_function,
  X_test = mnist.test.images,
  Y_test = mnist.test.labels)

epoch: 0000 loss = 58.495139
epoch: 0001 loss = 13.683967
epoch: 0002 loss = 8.473825
epoch: 0003 loss = 5.986947
epoch: 0004 loss = 4.450969
epoch: 0005 loss = 3.510556
epoch: 0006 loss = 2.800619
epoch: 0007 loss = 2.301075
epoch: 0008 loss = 1.903534
epoch: 0009 loss = 1.598667
epoch: 0010 loss = 1.344912
epoch: 0011 loss = 1.132986
epoch: 0012 loss = 0.981053
epoch: 0013 loss = 0.831587
epoch: 0014 loss = 0.726735
epoch: 0015 loss = 0.615635
epoch: 0016 loss = 0.534614
epoch: 0017 loss = 0.466414
epoch: 0018 loss = 0.406133
epoch: 0019 loss = 0.354212
epoch: 0020 loss = 0.305587
epoch: 0021 loss = 0.278697
epoch: 0022 loss = 0.237243
epoch: 0023 loss = 0.213829
epoch: 0024 loss = 0.181124
epoch: 0025 loss = 0.154062
epoch: 0026 loss = 0.143423
epoch: 0027 loss = 0.119653
epoch: 0028 loss = 0.104981
epoch: 0029 loss = 0.091400
epoch: 0030 loss = 0.072351
epoch: 0031 loss = 0.073604
epoch: 0032 loss = 0.057809
epoch: 0033 loss = 0.052322
epoch: 0034 loss = 0.045314
epoch: 0035 loss =

In [12]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD


Using TensorFlow backend.


In [0]:
num_layers = 2
num_neurons = []
for i in range(num_layers):
  num_neurons.append(256)
learning_rate = 0.01
n_epochs = 50
batch_size = 100

In [0]:
model = Sequential()

In [0]:
model.add(Dense(units=num_neurons[0], activation='relu', input_shape=(num_inputs,)))
model.add(Dense(units=num_neurons[1], activation='relu'))

In [0]:
model.add(Dense(units=num_outputs, activation='softmax'))


In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               200960    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=learning_rate),
  metrics=['accuracy'])

In [19]:
model.fit(X_train, Y_train,
  batch_size=batch_size,
  epochs=n_epochs)

Epoch 1/50
55000/55000 [==============================] - 4s 73us/step - loss: 1.1041 - acc: 0.7537
Epoch 2/50
55000/55000 [==============================] - 4s 70us/step - loss: 0.4398 - acc: 0.8830
Epoch 3/50
55000/55000 [==============================] - 4s 69us/step - loss: 0.3548 - acc: 0.9007
Epoch 4/50
55000/55000 [==============================] - 4s 70us/step - loss: 0.3161 - acc: 0.9107
Epoch 5/50
55000/55000 [==============================] - 4s 69us/step - loss: 0.2906 - acc: 0.9175
Epoch 6/50
55000/55000 [==============================] - 4s 69us/step - loss: 0.2710 - acc: 0.9231
Epoch 7/50
55000/55000 [==============================] - 4s 69us/step - loss: 0.2547 - acc: 0.9285
Epoch 8/50
55000/55000 [==============================] - 4s 68us/step - loss: 0.2401 - acc: 0.9325
Epoch 9/50
55000/55000 [==============================] - 4s 68us/step - loss: 0.2276 - acc: 0.9363
Epoch 10/50
55000/55000 [==============================] - 4s 66us/step - loss: 0.2165 - acc: 0.9392

In [20]:
score = model.evaluate(X_test, Y_test)
print('\n Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 0s 48us/step

 Test loss: 0.08758188138827681
Test accuracy: 0.9725
